In [19]:
!nvidia-smi

Sun Aug  4 07:14:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0              30W /  70W |    657MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [20]:
import numpy as np

In [21]:
!pip install sentence-transformers

In [22]:
from sentence_transformers import SentenceTransformer
model_name = "multi-qa-mpnet-base-dot-v1"
embedding_model = SentenceTransformer(model_name)


In [23]:
import pandas as pd

In [24]:
import pandas as pd
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'

# Read the dataset
df = pd.read_csv(url)

# Use only the first 300 documents
df = df.iloc[:300]

In [25]:
print(df.columns)

Index(['answer_llm', 'answer_orig', 'document', 'question', 'course'], dtype='object')


In [26]:
answer_llm = df.iloc[0].answer_llm
embeddings = embedding_model.encode(answer_llm)

In [27]:
print(embeddings[0])

-0.42244664


What's the first value of the resulting vector?

**-0.42**
-0.22
-0.02
0.21

In [38]:
import numpy as np
evaluations = []
for index, row in df.iterrows():
    answer_llm = row['answer_llm']
    answer_orig = row['answer_orig']

    embedding_llm = embedding_model.encode(answer_llm)
    embedding_orig = embedding_model.encode(answer_orig)

    cosine_similarity = np.dot(embedding_llm, embedding_orig)

    evaluations.append(cosine_similarity)

# Calculate the 75th percentile of the cosine similarities
percentile_75 = np.percentile(evaluations, 75)

# Print the 75th percentile
print(f"The 75th percentile of the cosine similarities is: {percentile_75}")

The 75th percentile of the cosine similarities is: 31.674304962158203


What's the 75% percentile of the score?

21.67
**31.67**
41.67
51.67

Answer:** 31.67**

In [30]:
def normalize(v):
    norm = np.sqrt((v * v).sum())
    return v / norm

In [57]:
import numpy as np

# Initialize an empty list to store the cosine similarity scores
evaluations = []

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    answer_llm = row['answer_llm']
    answer_orig = row['answer_orig']

    # Create embeddings for both answers using the embedding model
    embedding_llm = embedding_model.encode(answer_llm)
    embedding_orig = embedding_model.encode(answer_orig)

    # Normalize the embeddings to ensure they have a unit length
    embedding_llm_norm = embedding_llm / np.linalg.norm(embedding_llm)
    embedding_orig_norm = embedding_orig / np.linalg.norm(embedding_orig)

    # Compute the dot product (cosine similarity) between the normalized embeddings
    cosine_similarity = np.dot(embedding_llm_norm, embedding_orig_norm)

    # Append the result to the evaluations list
    evaluations.append(cosine_similarity)

# Calculate the 75th percentile of the cosine similarities
percentile_75 = np.percentile(evaluations, 75)

# Print the 75th percentile
print(f"The 75th percentile of the cosine similarities is: {percentile_75}")

The 75th percentile of the cosine similarities is: 0.8362347185611725


What's the 75% cosine in the scores?

0.63
0.73
**0.83**
0.93

Answer: **0.83**

In [58]:
# !pip install rouge

In [59]:
!pip install rouge-score


In [60]:
# Install the necessary package
# !pip install rouge-score

# Import the necessary libraries
import pandas as pd
from rouge_score import rouge_scorer


github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)
df = df.iloc[:300]
rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
r = df.iloc[10]
scores = rouge_scorer.score(r['answer_llm'], r['answer_orig'])
print(f"The F1 score for ROUGE-1 is: {scores['rouge1'].fmeasure}")


The F1 score for ROUGE-1 is: 0.5063291139240506


In [61]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

r = df.iloc[10]
answer_llm = r['answer_llm']
answer_orig = r['answer_orig']

In [62]:
scores = scorer.score(answer_llm, answer_orig)
rouge_1_f1 = scores['rouge1'].fmeasure
print(f"F1 score for rouge-1: {rouge_1_f1}")

F1 score for rouge-1: 0.5063291139240506


In [63]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

r = df.iloc[10]
answer_llm = r['answer_llm']
answer_orig = r['answer_orig']

scores = scorer.score(answer_llm, answer_orig)

rouge_1_f = scores['rouge1'].fmeasure
rouge_2_f = scores['rouge2'].fmeasure
rouge_l_f = scores['rougeL'].fmeasure

average_f_score = (rouge_1_f + rouge_2_f + rouge_l_f) / 3

print(f"Rouge-1 F-score: {rouge_1_f}")
print(f"Rouge-2 F-score: {rouge_2_f}")
print(f"Rouge-L F-score: {rouge_l_f}")
print(f"Average F-score: {average_f_score}")

Rouge-1 F-score: 0.5063291139240506
Rouge-2 F-score: 0.3116883116883117
Rouge-L F-score: 0.37974683544303806
Average F-score: 0.3992547536851334


In [64]:

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_1_f = []
rouge_2_f = []
rouge_l_f = []


for index, row in df.iterrows():
    scores = scorer.score(row['answer_llm'], row['answer_orig'])
    rouge_1_f.append(scores['rouge1'].fmeasure)
    rouge_2_f.append(scores['rouge2'].fmeasure)
    rouge_l_f.append(scores['rougeL'].fmeasure)


rouge_scores_df = pd.DataFrame({
    'rouge-1_f': rouge_1_f,
    'rouge-2_f': rouge_2_f,
    'rouge-l_f': rouge_l_f
})


average_rouge_2 = rouge_scores_df['rouge-2_f'].mean()

print(f"The average ROUGE-2 score across all records is: {average_rouge_2}")

The average ROUGE-2 score across all records is: 0.2880237296538236


In [65]:
import sys
!{sys.executable} -m pip install rouge
from rouge import Rouge

In [55]:
from rouge import Rouge

# URL of the dataset
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'

# Read the dataset
df = pd.read_csv(url)

# Use only the first 300 documents
df = df.iloc[:300]

# Initialize the ROUGE scorer
rouge_scorer = Rouge()

# Get the answers at index 10
r = df.iloc[10]

# Compute the ROUGE scores
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

# Print the F1 score for ROUGE-1
print(f"The F1 score for ROUGE-1 is: {scores['rouge-1']['f']}")


The F1 score for ROUGE-1 is: 0.45454544954545456


What's the F score for rouge-1?

0.35
**0.45**
0.55
0.65

Answer: **0.45**

In [66]:
from rouge import Rouge

rouge = Rouge()

r = df.iloc[10]
answer_llm = r['answer_llm']
answer_orig = r['answer_orig']

scores = rouge.get_scores(answer_llm, answer_orig)[0]

rouge_1_f = scores['rouge-1']['f']
rouge_2_f = scores['rouge-2']['f']
rouge_l_f = scores['rouge-l']['f']

average_f_score = (rouge_1_f + rouge_2_f + rouge_l_f) / 3

print(f"Rouge-1 F-score: {rouge_1_f}")
print(f"Rouge-2 F-score: {rouge_2_f}")
print(f"Rouge-L F-score: {rouge_l_f}")
print(f"Average F-score: {average_f_score}")

Rouge-1 F-score: 0.45454544954545456
Rouge-2 F-score: 0.21621621121621637
Rouge-L F-score: 0.393939388939394
Average F-score: 0.35490034990035496


 Average F-score between rouge-1, rouge-2 and rouge-L for the same record from Q4 task

**0.35**
0.45
0.55
0.65

In [67]:
rouge_scorer = Rouge()


rouge_1_f = []
rouge_2_f = []
rouge_l_f = []


for index, row in df.iterrows():
    scores = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]
    rouge_1_f.append(scores['rouge-1']['f'])
    rouge_2_f.append(scores['rouge-2']['f'])
    rouge_l_f.append(scores['rouge-l']['f'])


rouge_scores_df = pd.DataFrame({
    'rouge-1_f': rouge_1_f,
    'rouge-2_f': rouge_2_f,
    'rouge-l_f': rouge_l_f
})


average_rouge_2 = rouge_scores_df['rouge-2_f'].mean()


print(f"The average ROUGE-2 score across all records is: {average_rouge_2}")

The average ROUGE-2 score across all records is: 0.20696501983423318


the score for all the records and create a dataframe from them.

What's the average rouge_2 across all the records?

0.10
**0.20**
0.30
0.40